# Preparing the Sentinel-1 Data

### Import libraries

In [ ]:
import cv2
import os
import numpy as np
import imageio
from osgeo import gdal, osr
import multiprocessing

### Removal of infinity

The cell takes TIF image files from the input_folder, processes them to remove extreme values (positive and negative infinity), and saves the modified images to the output_folder. It also updates the geospatial information (coordinate system and projection) of the output TIF files to match that of the input files.

In [ ]:
input_folder = ""
output_folder = ""

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for filename in os.listdir(input_folder):
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        input_tif = os.path.join(input_folder, filename)
        output_tif = os.path.join(output_folder, filename)
        image = cv2.imread(input_tif, cv2.IMREAD_UNCHANGED)
        if image is not None:
            image[image == float('inf')] = 0
            image[image == -float('inf')] = 0
            cv2.imwrite(output_tif, image)
            print(f"TIF file '{filename}' with extreme values removed has been saved.")
            input_dataset = gdal.Open(input_tif)
            geo_transform = input_dataset.GetGeoTransform()
            projection = input_dataset.GetProjection()
            output_dataset = gdal.Open(output_tif, gdal.GA_Update)
            output_dataset.SetGeoTransform(geo_transform)
            output_dataset.SetProjection(projection)
            output_dataset.GetRasterBand(1).WriteArray(image)

            print(f"Geospatial information written to the TIF file.")
        else:
            print(f"Failed to load TIF file '{filename}'.")
            
print("Processing complete.")



## Reduce speckle noise

This cell applies a Lee filter to TIF images found in a specified input folder, reducing speckle noise. It then saves the filtered images to an output folder while preserving their geospatial properties. The script utilizes multiprocessing for efficient processing and improves image quality for further analysis or visualization purposes.

In [ ]:
def set_geotransform_and_projection(output_image_path, input_image_path):
    input_file = gdal.Open(input_image_path)
    output_file = gdal.GetDriverByName('GTiff').Create(output_image_path, input_file.RasterXSize, input_file.RasterYSize, 1, gdal.GDT_Float32)
    output_file.SetGeoTransform(input_file.GetGeoTransform())
    output_file.SetProjection(input_file.GetProjection())
    return output_file

def lee_filter(image, window_size):
    rows, cols = image.shape
    filtered_image = np.zeros_like(image)
    margin = window_size // 2

    for i in range(margin, rows - margin):
        for j in range(margin, cols - margin):
            window = image[i - margin:i + margin + 1, j - margin:j + margin + 1]
            mean = np.mean(window)
            std_dev = np.std(window)
            var = std_dev ** 2
            if var == 0:
                filtered_image[i, j] = image[i, j]
            else:
                filtered_image[i, j] = mean + ((std_dev ** 2) / (var + (std_dev ** 2))) * (image[i, j] - mean)

    return filtered_image

def process_image(input_image_path, output_image_path, window_size):
    output_file = set_geotransform_and_projection(output_image_path, input_image_path)
    image = imageio.imread(input_image_path)
    filtered_image = lee_filter(image, window_size)
    output_file.GetRasterBand(1).WriteArray(filtered_image)
    output_file = None

if __name__ == "__main__":
    input_folder = ""
    output_folder = ""
    window_size = 7 

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pool = multiprocessing.Pool(processes=4)  
    jobs = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".tif"):
            input_image_path = os.path.join(input_folder, filename)
            output_image_path = os.path.join(output_folder, filename)
            job = pool.apply_async(process_image, (input_image_path, output_image_path, window_size))
            jobs.append(job)

    for job in jobs:
        job.get()

    pool.close()
    pool.join()


## Conversion of scale

This script traverses through a directory structure containing TIF image files, converting their pixel values to decibels (dB) using a logarithmic scale. It then saves the processed images in a new directory structure, appending "_db.tif" to the filenames. Finally, it prints a message indicating the completion of the processing.

In [ ]:
root_folder = ""
output_folder = ""

def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    tif_files = [file for file in os.listdir(input_folder) if file.endswith(".tif")]
    for tif_file in tif_files:
        input_file_path = os.path.join(input_folder, tif_file)
        output_subfolder = os.path.join(output_folder, os.path.basename(input_folder))
        output_file_path = os.path.join(output_subfolder, os.path.splitext(tif_file)[0] + "_db.tif")

        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)

        input_ds = gdal.Open(input_file_path)
        if input_ds is None:
            print(f"Cannot open input file: {tif_file}")
            continue

        input_data = input_ds.ReadAsArray()
        decibel_data = 10 * np.log10(input_data)
        output_ds = gdal.GetDriverByName("GTiff").Create(
            output_file_path,
            input_ds.RasterXSize,
            input_ds.RasterYSize,
            1,
            gdal.GDT_Float32
        )

        output_ds.SetGeoTransform(input_ds.GetGeoTransform())
        output_ds.SetProjection(input_ds.GetProjection())
        output_ds.GetRasterBand(1).WriteArray(decibel_data)

        input_ds = None
        output_ds = None

for root, dirs, files in os.walk(root_folder):
    for directory in dirs:
        input_folder_path = os.path.join(root, directory)
        output_folder_path = os.path.join(output_folder, os.path.basename(root))
        process_folder(input_folder_path, output_folder_path)

print("Processing completed.")
